# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')


# Import API key
from api_keys import geoapify_key

In [12]:
pip install cartopy geoviews pyproj

  Using cached pyshp-2.3.1-py2.py3-none-any.whl.metadata (55 kB)
INFO: pip is looking at multiple versions of panel to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --- ------------------------------------ 0.9/11.0 MB 19.2 MB/s eta 0:00:01
   --------- ------------------------------ 2.6/11.0 MB 27.1 MB/s eta 0:00:01
   --------------- ------------------------ 4.3/11.0 MB 30.8 MB/s eta 0:00:01
   ---------------------- ----------------- 6.1/11.0 MB 32.4 MB/s eta 0:00:01
   ---------------------------- ----------- 7.7/11.0 MB 32.9 MB/s eta 0:00:01
   ---------------------------------- ----- 9.5/11.0 MB 33.6 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.0 MB 38.6 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 36.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/565.1 kB ? eta -:--:--
   -----------------------

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Chicago,41.8500,-87.6500,10.58,46,100,3.60,US,1733789701
1,1,Dallas,32.7668,-96.7836,18.10,65,20,4.63,US,1733789476
2,2,São Paulo,-23.5475,-46.6361,23.38,85,100,5.81,BR,1733789528
3,3,Buenos Aires,-34.6132,-58.3772,21.64,50,49,6.26,AR,1733789684


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", 
)

# Display the map
hvplot.save(humidity_map, "output_data/humidity_map.html")
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_lats_df = city_data_df[(city_data_df['Lat'] >= -90) & (city_data_df['Lat'] <= 45)]

# Drop any rows with null values
ideal_weather_city_df = cities_narrow_lats_df.dropna()

# Display sample data
ideal_weather_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Chicago,41.8500,-87.6500,10.58,46,100,3.60,US,1733789701
1,1,Dallas,32.7668,-96.7836,18.10,65,20,4.63,US,1733789476
2,2,São Paulo,-23.5475,-46.6361,23.38,85,100,5.81,BR,1733789528
3,3,Buenos Aires,-34.6132,-58.3772,21.64,50,49,6.26,AR,1733789684


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city_df[['City', 'Country','Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Chicago,US,41.8500,-87.6500,46,
1,Dallas,US,32.7668,-96.7836,65,
2,São Paulo,BR,-23.5475,-46.6361,85,
3,Buenos Aires,AR,-34.6132,-58.3772,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
params = {
    "limit":limit,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Chicago - nearest hotel: Jaslin Hotel
Dallas - nearest hotel: Lorenzo Hotel
São Paulo - nearest hotel: Hotel Cruz de Malta
Buenos Aires - nearest hotel: Hotel City Baires


,City,Country,Lat,Lng,Humidity,Hotel Name
0,Chicago,US,41.8500,-87.6500,46,Jaslin Hotel
1,Dallas,US,32.7668,-96.7836,65,Lorenzo Hotel
2,São Paulo,BR,-23.5475,-46.6361,85,Hotel Cruz de Malta
3,Buenos Aires,AR,-34.6132,-58.3772,50,Hotel City Baires


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Lat",
    scale = 1,
    color = "City", 
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hvplot.save(hotel_map, "output_data/hotel_map.html")
hotel_map 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)